In [1]:
import torch
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
from tqdm import tqdm

from dataset import MyData
from model import Net
from config import device
from helper import downscale_map, open_img_as_array

PATH_TO_DATASET = './CloC/'
IMG_SIZE = (217, 334)


Selected devie is cuda:0


## Test


### Training loop

In [7]:
with open('results2.txt', 'w') as f:
    for path_to_img in tqdm(glob(PATH_TO_DATASET + '*.png')[:100]):
        img_name = path_to_img[len(PATH_TO_DATASET):]
        f.write(f'{img_name}\n')
        normal_img_array = 1 - open_img_as_array(path_to_img) // 255
        steps = [20, 15, 10, 1]
        map_cascade = [downscale_map(normal_img_array, step)
                       for step in steps]

        for i, cas_map in enumerate(map_cascade):
            dataset = MyData(path_or_img=255 * cas_map, data_mode='img',
                             mode_3d='cylinder', radius=1, reduce_fctor=1, need_info=False)
            dataset_list = [dataset]
            model = Net(dataset_list=dataset_list, lr=1e-3)
            model.to(device)
            if i:
                model.load_state_dict(torch.load(
                    f'./state_dict/GlobalCas/{inum}/{i}_CloC.pt'))
            model.start_training(
                num_epochs=15e+3, my_weight=0.1, need_plot=False, need_save=False)
            inum = img_name.split('.')[0]
            model.save_state_dict(
                f'./state_dict/GlobalCas/{inum}/{i+1}_CloC.pt')
        mse, f1, f2 = model.test_model_(threshold=0.15)
        f.write(f'MSE: {mse:.2f} F1: {f1:.2f}, {f2:.2f}\n\n')


100%|██████████| 100/100 [9:16:13<00:00, 333.74s/it] 


### Metrics analysis

In [10]:
df = pd.DataFrame(columns=['name', 'mse_pretrained', 'f1_pretrained', 'f2_pretrained', 'mse_random', 'f1_random', 'f2_random'])
with open('results.txt', 'r') as f:
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        if line[-4:] == '.png':
            name = line
            df = df.append({'name': name}, ignore_index=True)
        elif line[:2] == 'Pr':
            mse_pretrained, f1_pretrained, f2_pretrained = map(float, f.readline().split()[1:]), map(float, f.readline().split()[1:])
            df.loc[df['name'] == name, ['mse_pretrained', 'f1_pretrained', 'f2_pretrained']] = mse_pretrained, f1_pretrained, f2_pretrained
        elif line[:2] == 'Ra':
            mse_random, f1_random, f2_random = map(float, f.readline().split()[1:]), map(float, f.readline().split()[1:])
            df.loc[df['name'] == name, ['mse_random', 'f1_random', 'f2_random']] = mse_random, f1_random, f2_random

df.to_excel('results.xlsx', index=False)

C:\Users\waclo\AppData\Local\Temp\ipykernel_16100\1068870.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'name': name}, ignore_index=True)


ValueError: not enough values to unpack (expected 3, got 2)